In [151]:
import subprocess
import time
import difflib
from pathlib import Path

import pandas as pd

In [152]:
def print_diff(string1: str, string2: str) -> None:
    """
    Prints the diff between two strings in a human-readable format.

    Args:
        string1 (str): The first string to compare.
        string2 (str): The second string to compare.

    Returns:
        None
    """
    diff = difflib.unified_diff(
        string1.splitlines(keepends=True),
        string2.splitlines(keepends=True),
        fromfile="String1",
        tofile="String2",
        lineterm="",
    )
    print("".join(diff))


In [153]:
def build_dtree():
    subprocess.run(["cargo", "build", "--release"])
    
build_dtree()

    Finished `release` profile [optimized] target(s) in 0.06s


In [154]:
def run_command(command: list[str]) -> tuple[float, int, str]:
    """
    Runs a command and captures execution time, peak memory usage, and output.
    
    Args:
        command (list[str]): Command to execute as a list of arguments.

    Returns:
        tuple[float, int, str]: Execution time (seconds), peak memory usage (KB), and output.
    """
    start_time = time.time()
    process = subprocess.Popen(
        command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True
    )
    stdout, stderr = process.communicate()
    execution_time = time.time() - start_time

    # Parse memory usage from /usr/bin/time format
    memory_usage = 0
    if stderr:
        for line in stderr.splitlines():
            if "maxresident" in line:
                memory_usage = int(line.split()[-1])

    return execution_time, memory_usage, stdout

In [155]:
def benchmark_programs(test_dirs: list[str], my_program: str, output_file: str) -> None:
    """
    Benchmarks the user's program and the `tree` command across multiple directories
    with various argument combinations.

    Args:
        test_dirs (list[str]): List of directories to benchmark on.
        my_program (str): Path to the user's tree-like program.
        output_file (str): File path to save the benchmark results.

    Returns:
        None
    """
    results = []

    # Define argument combinations to test
    argss = [
        [],  # Default arguments
        ["-L", "2"], 
        ["-a"],  # Show hidden files
        ["-L", "3", "-a", "-f"],  # Combination of all flags
    ]


    for directory in test_dirs:
        print(f"Benchmarking on: {directory}")

        for args in argss:
            # Run my_program with arguments
            my_command = [my_program] + args + [directory]
            my_time, my_memory, my_output = run_command(my_command)

            # Run tree command with arguments
            tree_command = ["tree"] + args + [directory]
            tree_time, tree_memory, tree_output = run_command(tree_command)

            # Validate outputs
            output_match = "Match" if my_output.lower().strip() == tree_output.lower().strip() else "Mismatch"
            
            if output_match == "Mismatch":
                print_diff(my_output, tree_output)
            
            
            
            # Save results
            results.append({
                "Directory": directory,
                "Program": "dtree",
                "Arguments": " ".join(args),
                "Time(s)": my_time,
                "Memory(KB)": my_memory,
                "OutputMatch": output_match,
            })

            results.append({
                "Directory": directory,
                "Program": "tree",
                "Arguments": " ".join(args),
                "Time(s)": tree_time,
                "Memory(KB)": tree_memory,
                "OutputMatch": "-",
            })

    # Save results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    print(f"Benchmark results saved to {output_file}")


In [156]:
# List of directories to benchmark on
test_directories = [
    str(Path.cwd().parent),
    # str(Path().home())
    ]

# Path to the user's tree-like program
user_program_path = str(Path().cwd().parent / "target/release/dtree")

# Output CSV file
output_csv = str(Path.cwd() / "results.csv")

# Run the benchmark
benchmark_programs(test_directories, user_program_path, output_csv)

Benchmarking on: /Users/roman/Repos/dtree
--- String1+++ String2@@ -1,935 +1,935 @@-.
-├── Cargo.lock
-├── Cargo.toml
-├── benchmarks
-│   ├── benchmark.ipynb
-│   └── results.csv
-├── dtree_output.txt
-├── src
-│   ├── dtree.rs
-│   └── main.rs
-├── target
-│   ├── CACHEDIR.TAG
-│   ├── debug
-│   │   ├── build
-│   │   │   ├── libc-e5564a49dda7de76
-│   │   │   │   ├── invoked.timestamp
-│   │   │   │   ├── out
-│   │   │   │   ├── output
-│   │   │   │   ├── root-output
-│   │   │   │   └── stderr
-│   │   │   ├── libc-f5a7fa853877e145
-│   │   │   │   ├── build-script-build
-│   │   │   │   ├── build_script_build-f5a7fa853877e145
-│   │   │   │   └── build_script_build-f5a7fa853877e145.d
-│   │   │   ├── lock_api-c33d0c5cab3b8fe1
-│   │   │   │   ├── build-script-build
-│   │   │   │   ├── build_script_build-c33d0c5cab3b8fe1
-│   │   │   │   └── build_script_build-c33d0c5cab3b8fe1.d
-│   │   │   ├── lock_api-d14f2df5ddaaaf9c
-│   │   │   │   ├── invoked.timestamp
-│   │   │   │   ├